In [9]:
import unittest

In [10]:
def load_data(n = None):
    # Default dataset
    males = {'0': ['7', '5', '6', '4'],
             '1': ['5', '4', '6', '7'],
             '2': ['4', '5', '6', '7'],
             '3': ['4', '5', '6', '7']}

    females = {'4': ['0', '1', '2', '3'],
               '5': ['0', '1', '2', '3'],
               '6': ['0', '1', '2', '3'],
               '7': ['0', '1', '2', '3']}

    # Dataset 1
    if n == 0:
        males = {'k': ['b', 'c', 'a'],
                 'l': ['a', 'c', 'b'],
                 'm': ['a', 'b', 'c']}

        females = {'a': ['k', 'l', 'm'],
                   'b': ['l', 'm', 'k'],
                   'c': ['m', 'l', 'k']}            
    return males, females

In [11]:
def is_empty(n):
    return len(n) == 0

class StableMarriageTestCase(unittest.TestCase):
    males_1 = {'0': ['7', '5', '6', '4'],
        '1': ['5', '4', '6', '7'],
        '2': ['4', '5', '6', '7'],
        '3': ['4', '5', '6', '7']}

    females_1 = {'4': ['0', '1', '2', '3'],
          '5': ['0', '1', '2', '3'],
          '6': ['0', '1', '2', '3'],
          '7': ['0', '1', '2', '3']}

    males_2 = {'k': ['b', 'c', 'a'],
            'l': ['a', 'c', 'b'],
            'm': ['a', 'b', 'c']}

    females_2 = {'a': ['k', 'l', 'm'],
              'b': ['l', 'm', 'k'],
              'c': ['m', 'l', 'k']}   
    
    def test_load_default_data(self):
        males, females = load_data()
        self.assertEqual(males, self.males_1)
        self.assertEqual(females, self.females_1)

    def test_load_data(self):
        males, females = load_data(0)
        self.assertEqual(males, self.males_2)
        self.assertEqual(females, self.females_2)
    
    def test_match_one(self):
        male_data, female_data = load_data()
        matches = Matches(male_data, female_data)
        expected = [frozenset({'0', '7'}),
                    frozenset({'1', '5'}),
                    frozenset({'2', '4'}),
                    frozenset({'3', '6'})]
        got = matches.match()
        self.assertEqual(expected, got)
    
    def test_match_two(self):
        male_data, female_data = load_data(0)
        matches = Matches(male_data, female_data)
        expected = [frozenset({'k', 'c'}), 
                    frozenset({'a', 'l'}), 
                    frozenset({'b', 'm'})]
        got = matches.match()
        self.assertEqual(expected, got)
        

if __name__ == '__main__':
    unittest.main(argv = ['ignore-first-argv'], exit = False)

....
----------------------------------------------------------------------
Ran 4 tests in 0.004s

OK


In [12]:
# output = {}

# epochs = len(males.keys())

# proposer = {}


# def match(i):
#     for male in males:
#         if male in proposer:
#             if proposer[male]:
#                 continue
#         choice = males[male][i]
#         print('guy {} pick girl {}'.format(male, choice))
#         if choice in output:
#             print('girl {} is choosen'.format(choice))
#             picked_male = output[choice]
#             if females[choice].index(picked_male) < females[choice].index(male):
#                 # Smaller index means higher preference
#                 print('girl {} picked {} instead of {}'.format(choice, picked_male, male))
#                 output[choice] = picked_male
#                 proposer[male] = False
#                 proposer[picked_male] = True
#             else:
#                 proposer[male] = True
#                 proposer[picked_male] = False
#         else:
#             output[choice] = male
#             proposer[male] = True
#     print(output, proposer)

# while not all(proposer.values()) or len(proposer.values()) != epochs:
#     for i in range(epochs):
#         match(i)

In [13]:
class Male:
    def __init__(self, name, preference):
        self.name = name
        self.preference = preference
        self.is_matched = False
        self.partner = None
        self.index = 0 # Index to hold the position of the proposed wives
    
    def propose(self):
        female = self.preference[self.index]
        self.index += 1
        return female
    
    def is_available(self):
        return not self.is_matched

    def __str__(self):
        if self.is_matched:
            return '{} is matched to {}'.format(self.name, self.partner.name)
        return '{} is single'.format(self.name)

In [14]:
class Female:
    def __init__(self, name, preference):
        self.name = name
        self.preference = preference
        self.is_matched = False
        self.partner = None
    
    def is_available(self):
        return not self.is_matched

    def rank(self, m1, m2):
        m1_score = self.preference.index(m1)
        m2_score = self.preference.index(m2)

        # Lower index means higher preference
        return m1 if m1_score < m2_score else m2

    def __str__(self):
        if self.is_matched:
            return '{} is matched to {}'.format(self.name, self.partner.name)
        return '{} is single'.format(self.name)

In [15]:
class Matches:
    def __init__(self, males, females):
        self.matches = {}

        self.choices = len(males)
        self.males = list(males.keys())
        self.females = list(females.keys())
        self.match_count = 0
        
        for i in males:
            self.matches[i] = Male(i, males[i])

        for j in females:
            self.matches[j] = Female(j, females[j])
        
    def engage(self, male_name, female_name):
        """Engage the male to the female if both of them are still available
        
        Args:
            male_name (str): The name of the male
            female_name (str): The name of the female
        """
        male = self.matches[male_name]
        female = self.matches[female_name]
        
        male.is_matched = True
        male.partner = female
        
        female.is_matched = True
        female.partner = male
        
        self.match_count += 1
    
    def breakup(self, male_name, female_name):
        """Break the engagement for both the male and female
        
        Args:
            male_name (str): The name of the male
            female_name (str): The name of the female
        """
        male = self.matches[male_name]
        female = self.matches[female_name]
        
        male.is_matched = False
        male.partner = None
        
        female.is_matched = False
        female.partner = None

        self.match_count -= 1
    
    def match(self):
        """Recursively match the male and female as long as they 
        have not been matched. The matching ends once each male is
        paired with a female"""
        for m in self.males:
            male = self.matches[m]
            if not male.is_available():
                continue

            female = self.matches[male.propose()]
            if female.is_available():
                self.engage(male.name, female.name)
            else:
                male_name = female.rank(male.name, female.partner.name)
                self.breakup(female.partner.name, male_name)
                self.engage(male_name, female.name)
        if self.match_count < self.choices:
            self.match()

        return self.sets()
    
    def sets(self):
        """Returns the male-female pairs that have been matched
        
        Returns:
            matches (:obj:`list` of :obj:`str`): The unique pairs of male-female
        """
        matches = {}
        for i in self.matches:
            match = self.matches[i]
            matches[frozenset([match.name, match.partner.name])] = True
        return list(matches.keys())

In [16]:
male_data, female_data = load_data(1)
matches = Matches(male_data, female_data)
matches.match()

[frozenset({'0', '7'}),
 frozenset({'1', '5'}),
 frozenset({'2', '4'}),
 frozenset({'3', '6'})]